In [2]:
import sys
sys.path.append("../..")
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import random
%matplotlib inline

In [3]:
import torch
from torch import nn
from torch import distributions
from torch.nn.parameter import Parameter
from new_train import train_nis
from new_train import train_nisp_rnis
from nis_net import NISNet
from nis_net import NISPNet
from nis_net import RNISNet
from datetime import datetime
from new_kuramoto import KuramotoModel
t0 = datetime.now()


use_cuda = torch.cuda.is_available()
device = torch.device('cuda:0') if use_cuda else torch.device('cpu')
device

device(type='cuda', index=0)

In [ ]:
seed = 2050
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

kura_train = KuramotoModel(steps=20000, dt=0.01, sz=16, groups=2, coupling=30)
timeserie, timeserie_next, _, _ = kura_train.simulate_oneserie()
timeserie.shape

In [ ]:
plt.figure(figsize=(12,5))
for i in range(16):
    plt.plot(timeserie[:,i])
plt.show()